In [ ]:
pip install yfinance

In [ ]:
pip install finance-datareader

In [3]:
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf #for Korean market
import FinanceDataReader as fdr #for US market
import datetime
import csv

In [4]:
def getStockCode(market):
    if market == 'kosdaq':
        url_market = 'kosdaqMkt'
    elif market == 'kospi':
        url_market = 'stockMkt'
    else:
        print('invalid market ')
        return
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=%s' % url_market
    df = pd.read_html(url, header=0)[0]
    return df

In [ ]:
# Retrieving KOSPI data

df = getStockCode('kospi')

fail = [] #Failed retrievals

for _, data in df.iterrows():
    code = str(data['종목코드'])
    code_len = len(code)
    new_code = "0"*(6-code_len)+code+".KS"
    try:  
        company_data = pdr.get_data_yahoo(new_code, start=data['상장일'], end="2021-10-29")
        company_data.to_csv('data/raw/kospi/'+data['회사명']+'.csv')
    except:
        print(data['회사명']+' retreival unsuccessful')
        fail.append(data['회사명'])

print(fail)

In [ ]:
# Retrieving NASDAQ data

df = fdr.StockListing('NASDAQ')

for _, data in df.iterrows():
    done = False
    dtstr = "1980-01-01"
    while not done:
        try:
            company_data = pdr.get_data_yahoo(data['Symbol'], start=dtstr, end="2021-10-29")
            done = True
        except:
            dt = datetime.datetime.strptime(dtstr,'%Y-%m-%d')
            newd = dt+datetime.timedelta(days=1)
            dtstr = newd.strftime("%Y-%m-%d")
    company_data.to_csv("data/raw/nasdaq/"+''.join(e for e in data['Name'] if e.isalnum())+".csv") # remove special characters from company name

In [ ]:
print('done')